In [1]:
import bentoml
import torch
import xgboost as xgb
from transformers import AutoTokenizer, AutoModel
model_name = "sentence-transformers/all-MiniLM-L6-v2"
# Loading from HuggingFace
tokenizer = AutoTokenizer.from_pretrained(model_name)
hf_model = AutoModel.from_pretrained(model_name)

/home/sath/airflow-docker/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# saved1 = bentoml.transformers.save_model(
#         "embedmodel",
#         hf_model,
#         custom_objects={"tokenizer": tokenizer}
# )
# saved2 = bentoml.transformers.save_model(
#         "embedmodel_tokenizer",
#         tokenizer,
#         custom_objects={"tokenizer": tokenizer}
# )

model = xgb.Booster()
model.load_model("/home/sath/airflow-docker/data/model_MiniLM-L6.json")
xgb_model = "xgb-model-v2"
bentoml.xgboost.save_model(xgb_model, model)


# print(f"Saved to BentoML: {saved1.tag}")
# print(f"Saved to BentoML: {saved2.tag}")

Model(tag="xgb-model-v2:jzqlmmxppshqh7yx", path="/tmp/bentoml-model-xgb-model-v2-3qnzoyo6")

In [15]:
# Get the model reference
model_ref = bentoml.models.get("embedmodel:latest")
tokenizer_ref = bentoml.models.get("embedmodel_tokenizer:latest")
print(f"Model path: {model_ref.path}")
print(f"Tokenizer path: {tokenizer_ref.path}")

Model path: /home/sath/bentoml/models/embedmodel/ckkfltxpo2qpwboj
Tokenizer path: /home/sath/bentoml/models/embedmodel_tokenizer/cmczyaxpo2qpwboj


In [16]:
loaded_tokenizer = AutoTokenizer.from_pretrained(tokenizer_ref.path)
loaded_model = AutoModel.from_pretrained(model_ref.path)

In [ ]:
# Testing inference
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

inputs = loaded_tokenizer("Hello world", return_tensors="pt", padding=True, truncation=True)
outputs = loaded_model(**inputs)
shape = outputs.last_hidden_state.shape
print(f"Output shape: {shape}")

✓ Output shape: torch.Size([1, 4, 384])


In [5]:
xgb_model = "xgb-model-v1"
xgb_ref = bentoml.xgboost.load_model(f"{xgb_model}:latest")